# Analyze Data
### Extract data infromation that can help us decide how to design the model and preprocess the data

In [1]:
import cv2
import matplotlib.pyplot as plt
import numpy as np 
import json

## Verify true fps match

In [17]:
# Load videos verify fps match 
cap_train = cv2.VideoCapture('./data/images/train/train.mp4')
cap_test = cv2.VideoCapture('./data/images/test/test.mp4')
cap_val = cv2.VideoCapture('./data/images/val/val.mp4')

print(f"train fps: {cap_train.get(cv2.CAP_PROP_FPS)}")
print(f"test fps: {cap_test.get(cv2.CAP_PROP_FPS)}")
print(f"val fps: {cap_val.get(cv2.CAP_PROP_FPS)}")

cap_train.release()
cap_test.release()
cap_val.release()

train fps: 30.0
test fps: 30.0
val fps: 30.0


In [14]:
RES_H = 640
RES_W = 640 
FPS = 30 

## Check annotations, instances, occurences, count (ground truth statistics)

### Define helper functions

In [2]:
# Extract necessary data from json
def load_json_data(path):
    file = open(path)
    data = json.load(file)
    file.close()
    return data

In [3]:
ID_BOLT = 1
ID_NUT = 2

In [4]:
# 1 - bolt, 2 - nut 
train_data = load_json_data('./data/annotations/instances_train.json')
train_annotations = train_data['annotations']
no_frames = len(train_data['images'])
print('Number of training frames: ',no_frames)

Number of training frames:  7200


In [24]:
# Frames with bolt instance (repetition allowed)
frames_with_bolt = [instance['image_id'] for instance in train_annotations if instance['category_id'] == ID_BOLT]
# Frames with nut instance (repetition allowed)
frames_with_nut = [instance['image_id'] for instance in train_annotations if instance['category_id'] == ID_NUT]
# Frames with nut both
frames_with_both = np.intersect1d(frames_with_bolt, frames_with_nut, assume_unique=False)

frames_with_bolt, frames_with_bolt_count = np.unique(frames_with_bolt, return_counts=True)
frames_with_nut, frames_with_nut_count = np.unique(frames_with_nut, return_counts=True)

### Percentage of videos with Bolts

In [27]:
print("True frame = frame containint the object of interest \n")
print(f'Percentage of frames with bolts: {round(len(frames_with_bolt)*100/no_frames)}%')
print("Bolts per true frame: {:.2f} \n".format(np.mean(frames_with_bolt_count)))

print(f'Percentage of frames with nuts: {round(len(frames_with_nut)*100/no_frames)}%')
print("Nuts per true frame: {:.2f} \n".format(np.mean(frames_with_nut_count)))

print(f'Percentage of frames with both: {round(len(frames_with_both)*100/no_frames)}%')

True frame = frame containint the object of interest 

Percentage of frames with bolts: 92%
Bolts per true frame: 2.23 

Percentage of frames with nuts: 50%
Nuts per true frame: 1.00 

Percentage of frames with both: 47%


### Analyze labeling quality

In [38]:
import os

In [2]:
# Draw the bbox around the images 
orig_dir = 'C:/Users/mirza/Desktop/stroma-tracking/data/experiment/frames/train'
target_dir = 'C:/Users/mirza/Desktop/stroma-tracking/data/experiment/frames_with_boxes/train'
LIMIT = 10

for i, annotation in enumerate(train_annotations):
    image_id = annotation['image_id']
    bbox = annotation['bbox']
    xmin = int(bbox[0])
    ymin = int(bbox[1])
    xmax = int(bbox[0] + bbox[2])
    ymax = int(bbox[1] + bbox[3])
    #print(xmin, ymin, xmax, ymax)
    
    if os.path.exists('{}/{:04d}.jpg'.format(target_dir,image_id)):
        image = cv2.imread('{}/{:04d}.jpg'.format(target_dir,image_id))
    else:
        image = cv2.imread('{}/{:04d}.jpg'.format(orig_dir,image_id))
        
    _ = cv2.rectangle(image, (xmin, ymin), (xmax, ymax), (0, 0, 255), 2)
    cv2.imwrite('{}/{:04d}.jpg'.format(target_dir,image_id), image)
    
    if i > 1000:
        break

In [ ]:
python predict_on_video.py ./data/images/test/test.mp4